In [11]:
import CalculatedFieldSubroutines as cfs

#

import numpy as np

import pandas as pd

#

import matplotlib.pyplot as plt

from pandasgui import show

#

import warnings

#

import os

In [12]:
warnings.filterwarnings( 'ignore' )

In [13]:
gmIDs = cfs.list_whitelisted_gmIDs()

topics = cfs.list_topics()

print( topics )

['/apollo/sensor/gnss/best/pose', '/apollo/drive/event', '/apollo/canbus/chassis', '/apollo/perception/traffic/light']


In [14]:
def CreatePreprocessedMovingDataFolder( window ): # window in seconds

    for index, gmID in enumerate( gmIDs ):

        chassis_df = cfs.retrieve_gmID_topic( gmID, '/apollo/canbus/chassis' )

        pose_df = cfs.retrieve_gmID_topic( gmID, '/apollo/sensor/gnss/best/pose' )

        traffic_df = cfs.retrieve_gmID_topic( gmID, '/apollo/perception/traffic/light' )

        #

        if ( traffic_df.empty == True ):

            print( f'{ gmID } has no traffic light data' )

            continue

        #

        chassis_df = chassis_df.sort_values( 'time' )

        pose_df = pose_df.sort_values( 'time' )

        traffic_df = traffic_df.sort_values( 'time' )

        #

        cfs.Index( chassis_df )

        #

        cfs.NormalizedTime( chassis_df )

        #

        cfs.BinaryDrivingMode( chassis_df )

        cfs.TernaryDrivingModeTransition( chassis_df )

        cfs.BinaryDisengagement( chassis_df )

        #

        cfs.Acceleration( chassis_df )

        #

        cfs.TernaryTurnSignal( chassis_df )

        #

        chassis_df = chassis_df.drop( [ 'drivingMode', 'TernaryDrivingModeTransition', 'signal.turnSignal' ], axis = 1 )

        #

        cfs.LatLonTotalStdDev( pose_df )

        #

        cfs.ProgressAlongRoute_v2( pose_df )

        #

        pose_df = pose_df.drop( [ 'heightMsl', 'groupMetadataID', 'latitudeStdDev', 'heightStdDev', 'longitudeStdDev', \
                              'PartitionNumber' ], axis = 1 )

        #

        cfs.BinaryContainLights( traffic_df )

        #

        traffic_df = traffic_df.drop( [ 'groupMetadataID', 'containLights', 'trafficLight', 'trafficLightDebug.distanceToStopLine', \
                                    'cameraId' ], axis = 1 )

        #

        cfs.ChassisBestPoseMatchedTime( chassis_df, pose_df )

        merged_df1 = pd.merge( chassis_df, pose_df, on = 'ChassisBestPoseMatchedTime', how = 'inner' )

        merged_df1 = merged_df1.rename( columns = { 'time_x' : 'time' } )

        merged_df1 = merged_df1.drop( [ 'ChassisBestPoseMatchedTime', 'time_y' ], axis = 1 )

        #

        cfs.ChassisBestPoseMatchedTime( merged_df1, traffic_df )

        merged_df2 = pd.merge( merged_df1, traffic_df, on = 'ChassisBestPoseMatchedTime', how = 'inner' )

        merged_df2 = merged_df2.rename( columns = { 'time_x' : 'time' } )

        merged_df2 = merged_df2.drop( [ 'ChassisBestPoseMatchedTime', 'time_y' ], axis = 1 )

        #

        merged_df2 = merged_df2[ [ 'Ind', 'groupMetadataID', 'time', 'NormalizedTime', 'BinaryDrivingMode', 'BinaryDisengagement', \
                               'latitude', 'longitude', 'ProgressAlongRoute', 'solStatus', 'solType', 'extendedSolutionStatus', \
                               'numSatsInSolution', 'speedMps', 'Acceleration', 'brakePercentage', 'throttlePercentage', \
                               'steeringPercentage', 'LatLonTotalStdDev', 'TernaryTurnSignal', 'BinaryContainLights' ] ]

        #

        if ( window > 0 ):

            cfs.MovingFunction_v2( merged_df2, 'time', window * 1e9, 'mean', [ 'speedMps', 'Acceleration', 'brakePercentage', \
                                                                        'throttlePercentage', 'steeringPercentage', \
                                                                        'LatLonTotalStdDev', 'TernaryTurnSignal', \
                                                                        'BinaryContainLights' ] )

            cfs.MovingFunction_v2( merged_df2, 'time', window * 1e9, 'stddev', [ 'speedMps', 'Acceleration', 'brakePercentage', \
                                                                          'throttlePercentage', 'steeringPercentage', \
                                                                          'LatLonTotalStdDev', 'TernaryTurnSignal', \
                                                                          'BinaryContainLights' ] )

        #

        filepath = f'{ cfs.origin_dir() }/Preprocessed_Moving_Data/{ window }sec_Window/{ gmID }'

        os.makedirs( filepath, exist_ok = True )

        #

        merged_df2.to_csv( f'{ filepath }/{ gmID }_{ window }sec.csv', index = False )

In [15]:
CreatePreprocessedMovingDataFolder( 0 )

CreatePreprocessedMovingDataFolder( 10 )

69ab88ec-dc17-11ee-a158-97f8443fd730 has no traffic light data
88a68dd8-eef9-11ee-9385-ef789ffde1d3 has no traffic light data
fc211bb2-efca-11ee-b966-fb353e7798cd has no traffic light data
69ab88ec-dc17-11ee-a158-97f8443fd730 has no traffic light data
88a68dd8-eef9-11ee-9385-ef789ffde1d3 has no traffic light data
fc211bb2-efca-11ee-b966-fb353e7798cd has no traffic light data
